In [1]:
import sys

sys.path.append("../")

from bunkatopics import Bunka
from bunkatopics.functions.clean_text import clean_tweet
from langchain.embeddings import HuggingFaceEmbeddings
import random
from datasets import load_dataset
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
dataset = load_dataset("rguo123/trump_tweets")["train"]
full_docs = dataset["content"]
full_docs = [clean_tweet(x) for x in full_docs]
full_docs = [x for x in full_docs if len(x)>50]
full_docs = random.sample(full_docs, 5000)

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
bunka = Bunka(embedding_model=embedding_model)
bunka.fit(full_docs)

from langchain.llms import OpenAI
open_ai_generative_model = OpenAI(openai_api_key = os.getenv('OPEN_AI_KEY'))





In [7]:
from pydantic import BaseModel
import typing as t 
from bunkatopics.functions.bourdieu_api import bourdieu_api
from bunkatopics.datamodel import TopicParam, TopicGenParam


class BourdieuQuery(BaseModel):
    x_left_words: t.List[str] = ["war"]
    x_right_words: t.List[str] = ["peace"]
    y_top_words: t.List[str] = ["men"]
    y_bottom_words: t.List[str] = ["women"]
    radius_size: float = 0.5


res = bourdieu_api(
            bunka.embedding_model,
            bunka.docs,
            bunka.terms,
            bourdieu_query = BourdieuQuery(),
            topic_param = TopicParam(n_clusters = 10),
            generative_ai_name=False,
            topic_gen_param= TopicGenParam(generative_model = open_ai_generative_model))

bourdieu_docs = res[0]
bourdieu_topics = res[1]

In [12]:
import json


file_path = "bourdieu_docs.json"
bourdieu_docs_json = [x.dict() for x in bourdieu_docs]

with open(file_path, 'w') as json_file:
    json.dump(bourdieu_docs_json, json_file)

   
file_path = "bourdieu_topics.json"
bourdieu_topics_json = [x.dict() for x in bourdieu_topics]
with open(file_path, 'w') as json_file:
    json.dump(bourdieu_topics_json, json_file)
